In [1]:
import pennylane as qml
import numpy as np

from QHyper.optimizers.cem import CEM
from QHyper.optimizers.random import Random
from QHyper.optimizers.qml_gradient_descent import QmlGradientDescent
from QHyper.solvers.qaoa.core import QAOA
from QHyper.optimizers.basinhopping import Basinhopping

## TSP problem

In [10]:
from QHyper.problems.tsp import TSPProblem

In [11]:
tsp = TSPProblem(
    number_of_cities=3,
)

In [13]:
solver = QAOA(
    problem=tsp,
    platform="pennylane",
    optimizer=QmlGradientDescent(qml.AdamOptimizer(stepsize=0.05), 200),
    layers=5,
    angles=[[0.5]*5, [0.5]*5],
    # mixer: str=,
    weights=[1, 1, 1],
    # hyperoptimizer=CEM(
    #     epochs=5,
    #     samples_per_epoch=100,
    #     elite_frac=0.1
    # ),
    # backend=
)

In [14]:
value, params, weights = solver.solve()

In [20]:
value

-0.2917606397447146

In [21]:
probs = solver.get_probs_func(weights)(params)

In [22]:
solver.check_results(probs)

-0.2917606397447146

In [8]:
solver.print_results(probs)

Key: 001100010 with probability 0.02379   | correct: True, value: 2.33220
Key: 010100001 with probability 0.02379   | correct: True, value: 2.33220
Key: 001010100 with probability 0.02379   | correct: True, value: 2.33220
Key: 100010001 with probability 0.02379   | correct: True, value: 2.33220
Key: 010001100 with probability 0.02379   | correct: True, value: 2.33220
Key: 100001010 with probability 0.02379   | correct: True, value: 2.33220
Key: 001110000 with probability 0.02196   | correct: False
Key: 000110001 with probability 0.02196   | correct: False
Key: 110000001 with probability 0.02196   | correct: False
Key: 110001000 with probability 0.02196   | correct: False
Key: 000001110 with probability 0.02196   | correct: False
Key: 001000110 with probability 0.02196   | correct: False
Key: 000010101 with probability 0.02144   | correct: False
Key: 000101010 with probability 0.02144   | correct: False
Key: 010101000 with probability 0.02144   | correct: False
Key: 101010000 with proba

## Knapsack problem

In [2]:
from QHyper.problems.knapsack import Knapsack, KnapsackProblem

In [3]:
knapsack_qaoa = KnapsackProblem(max_weight=1, items=[(1, 2), (1, 1)])

In [10]:

solver = QAOA(
    problem=knapsack_qaoa,
    platform="pennylane",
    optimizer=QmlGradientDescent(qml.AdamOptimizer(stepsize=0.05), 20),
    layers=5,
    angles=[[0.5]*5, [0.5]*5],
    # angles=[
        # [-1.55112026,  1.55112018, -5.58403279, -0.39782304, -3.94531351],
        # [-7.85398167,  2.35619439,  1.57079633, -3.14159267,  1.96349534]
    # ],
    weights=[2.72616422, 4.59436366],
    # weights=[2.5317127 , -1.26585641],
    # hyperoptimizer=CEM()
    # hyperoptimizer=Random(number_of_samples=10),
    hyperoptimizer=Basinhopping(niter=10, maxiter=2),
    # backend=
)
solver.solve()

(-1.6514748033728461,
 array([[ 0.64926115,  1.94194176,  1.60774618,  0.51222648, -0.56404832],
        [ 1.53541291,  0.69869471, -1.38010585, -0.54584033,  0.1252954 ]]),
 array([3.24109197, 4.806507  ]))

In [7]:
probs = solver.get_probs_func([2.5317127 , -1.26585641])([
        [-1.55112026,  1.55112018, -5.58403279, -0.39782304, -3.94531351],
        [-7.85398167,  2.35619439,  1.57079633, -3.14159267,  1.96349534]
    ])
solver.check_results(probs)

-1.9999999999998352

In [11]:
solver.counter

728

In [9]:
solver.counter

845

In [7]:
solver.counter

598

In [5]:
solver.counter

442

In [5]:
print(np.mean(results))
print(np.std(results))

-1.9289579553449545
0.05975628812245912


In [7]:
print(np.mean(results))
print(np.std(results))

-1.8427028195151585
0.40611565739824845


In [16]:
import multiprocessing as mp
import tqdm


def wrapper(params):
    solver = QAOA(
        problem=knapsack_qaoa,
        platform="pennylane",
        optimizer=QmlGradientDescent(qml.AdamOptimizer(stepsize=0.05), 200),
        layers=5,
        angles=[[params[0]]*5, [params[1]]*5],
        # weights=[2.72616422, 4.59436366, 4.46463072],
        weights=[1, 2],
        hyperoptimizer=Basinhopping(niters=),
        # hyperoptimizer=CEM(),
        # backend=
    )
    return solver.solve()

# weights = [1, 2, 2]
samples = 100

# angles = np.random.rand(samples, *np.array([[0.5]*5, [0.5]*5]).shape)
angles = np.random.rand(samples, 2)
        
with mp.Pool(processes=mp.cpu_count()) as p:
    results = list(tqdm.tqdm(
        p.imap(wrapper, angles), total=samples))

min_idx = np.argmin([result[0] for result in results])

100%|██████████| 100/100 [07:37<00:00,  4.58s/it]


In [18]:
results[min_idx]

(-1.5628950378594761,
 tensor([[0.79079128, 0.94395285, 0.74740293, 0.82420623, 0.77746254],
         [0.78169731, 1.79544682, 0.89165862, 1.11840858, 0.77773129]], requires_grad=True),
 [1, 2, 2])

In [13]:
results[min_idx] # 1000

(-1.9231064307286414,
 tensor([[ 0.67657702,  0.89682613,  1.06466848,  0.27857396,  0.5876597 ],
         [ 1.56435016,  0.81713401,  0.21456158,  0.18467615, -0.10288467]], requires_grad=True),
 [1, 2, 2])

In [11]:
results[min_idx] # 100

(-1.7636075482979467,
 tensor([[ 0.68864928,  0.88361648,  0.25525783,  0.63796315,  0.99512499],
         [ 1.52913702,  0.69534212, -0.12530048, -0.23152105, -0.24981503]], requires_grad=True),
 [1, 2, 2])

In [5]:
value, params, weights = solver.solve()

In [5]:
value

NameError: name 'value' is not defined

In [6]:
probs = solver.get_probs_func(weights)(params)

In [19]:
solver.check_results(probs)

-0.2917606397447146

In [14]:
solver.print_results(probs)

Key: 0010 with probability 0.25850   | correct: False
Key: 1001 with probability 0.14187   | correct: True, value: 2.00000
Key: 0101 with probability 0.11084   | correct: False
Key: 0000 with probability 0.08537   | correct: True, value: 0.00000
Key: 0011 with probability 0.06687   | correct: False
Key: 1000 with probability 0.05655   | correct: False
Key: 1111 with probability 0.05458   | correct: False
Key: 1011 with probability 0.05321   | correct: False
Key: 1101 with probability 0.04499   | correct: False
Key: 0111 with probability 0.03739   | correct: False
Key: 1010 with probability 0.03001   | correct: False
Key: 0100 with probability 0.02799   | correct: False
Key: 1100 with probability 0.01426   | correct: False
Key: 1110 with probability 0.00932   | correct: False
Key: 0110 with probability 0.00628   | correct: False
Key: 0001 with probability 0.00199   | correct: False


In [21]:
import pennylane as qml
import numpy as np
import multiprocessing as mp
import tqdm
import itertools

from QHyper.optimizers.cem import CEM
from QHyper.optimizers.random import Random
from QHyper.optimizers.shgo import Shgo
from QHyper.optimizers.qml_gradient_descent import QmlGradientDescent
from QHyper.solvers.qaoa.core import QAOA

from QHyper.problems.knapsack import Knapsack, KnapsackProblem

def compute(params):
    angles, weights = params
    knapsack_qaoa = KnapsackProblem(max_weight=1, items=[(1, 2), (1, 1)])

    solver = QAOA(
        problem=knapsack_qaoa,
        platform="pennylane",
        optimizer=QmlGradientDescent(qml.AdamOptimizer(stepsize=0.05), 200),
        layers=5,
        angles=[[angles[0]]*5, [angles[1]]*5],
        weights=weights,
    )

    value, params, weights = solver.solve()
    return value, params, weights

In [18]:
angles_samples = 3
weights_samples = 2

angles = np.random.rand(angles_samples, 2)
weights = 10*np.random.rand(weights_samples, 3) + 1


In [19]:
params = list(itertools.product(angles,weights))
params

[(array([0.88896497, 0.93709417]),
  array([3.15229747, 7.33439839, 7.87729378])),
 (array([0.88896497, 0.93709417]),
  array([ 4.50765762,  6.57945019, 10.34825826])),
 (array([0.31961787, 0.900045  ]),
  array([3.15229747, 7.33439839, 7.87729378])),
 (array([0.31961787, 0.900045  ]),
  array([ 4.50765762,  6.57945019, 10.34825826])),
 (array([0.11611688, 0.88807239]),
  array([3.15229747, 7.33439839, 7.87729378])),
 (array([0.11611688, 0.88807239]),
  array([ 4.50765762,  6.57945019, 10.34825826]))]

In [22]:

with mp.Pool(processes=mp.cpu_count()) as p:
    results = list(tqdm.tqdm(p.imap(compute, params), total=angles_samples*weights_samples))

min_idx = np.argmin([result[0] for result in results])
print(results[min_idx])


100%|██████████| 6/6 [00:21<00:00,  3.60s/it]

(-1.9769337352580112, tensor([[0.89102361, 0.86116821, 0.87637958, 0.94982761, 0.80780986],
        [0.69840863, 0.5480527 , 1.1515491 , 1.36703555, 0.38543498]], requires_grad=True), array([ 4.50765762,  6.57945019, 10.34825826]))
